In [1]:
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml

C:\Users\student\ANACONDA\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
url= "https://www.houseinrwanda.com/"

In [14]:
def house_urls(url, url_page):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    h5_tag= soup.select("h5")
    links = []
    for h5 in h5_tag:
        links.append(url + h5.find("a")["href"])
    return links

In [15]:
def navigate_pages(url, last_page):
    urls = []
    for i in range(0, last_page):
        urls.extend(house_urls(url, url+'?page=' + str(i)))
    return urls

In [6]:
def info(link_url):
    response = requests.get(link_url)
    soup = BeautifulSoup(response.text, 'lxml')
    prop_desc = soup.select('div.card-body')
    advert_lines = prop_desc[0].text.split('\n')

    advert_lines = [line for line in advert_lines if line]
    
    if advert_lines == []:
        advert_lines = prop_desc[1].text.split('\n')
        advert_lines = [line for line in advert_lines if line]
    
    advert_dict = {}
    advert_dict["Title"] = soup.find_all("title")[0].text
    for line in advert_lines:
        key_value = line.split(':')
        key = key_value[0].strip()
        value = key_value[1].strip()
        advert_dict[key] = value
    return advert_dict


In [7]:
def final_data(urls):
    data = []
    for link in urls:
        data.append(info(link))
    df = pd.DataFrame(data)
    return df

In [18]:
urls = navigate_pages(url=url, last_page=1)

In [19]:
urls[1]

'https://www.houseinrwanda.com//property/rent/apartment/kigali-apartment-rent-gisozi-belle-etoile-close-main-road'

In [22]:
data_csv = final_data(urls[1:3])

In [23]:
data_csv

,Title,Advert type,Price,Bedrooms,Total floors,Address,Property type,Bathrooms,Plot size,Furnished,Expiry date
0,Kigali Apartment for rent in Gisozi (Belle Eto...,Rent,600 Usd /month,3,1,"Kigali City, Gasabo, Gisozi",Apartment,3,,Yes,"June 01, 2023"
1,A furnished house for rent located in Kanombe ...,Rent,Price on request,5,,"Kigali City, Kicukiro, Kanombe",House,5,,Yes,"October 01, 2023"


In [ ]:
def extract_data_from_table(table, yr, team):
    """
    Given an html table for a single year, extract all the data
    Arguments:
    table --  html table retrieved using BS
    yr -- year under consideration
    team -- team/country name
    
    """
    
    # Get all rows from the table using the function: findChildren()
    # on the table object  with 'tr' as parameter
    rows = table.findChildren(['tr'])
    
    # List to hold the data 
    data = []
    
    # Loop through the rows and do the following:
    # 1. get cells from each row using function findChildren() with parameters 'td', 'th'
    # 2. Declare variables to hold information for player number, position and club
    # 3. Loop through eeach cell and do the following:
    # 1. Retrive text from the cell using text() function and remove 
    # "\n" with function strip()
    # 2. After some cleaning, set player number, position and club to text retrieved
    
    # first for loop (use row the counter variable) to keep with my variable naming below
    for row in rows:
        try:
            cells = row.findChildren(['td', 'th'])
            num, pos, name, club = None, None, None, None
            # second for loop use enumerate with i, cell as counter variables
            for i, cell in enumerate(cells):
                txt = cell.text.strip('\n')
                if i == 0:
                    try:
                        # convert to integer
                        num = int(txt)
                    except:
                        if txt == " ":
                            num = np.nan
                elif i == 1:
                    pos = txt
                elif i == 2:
                    name = txt
                elif i == 6:
                    club = txt
            # dictionary which holds the data we need for EventYear, team name, player number
            # position, name and club
            item = {"EventYear":yr, "Team":team, "Number":num, "Position": pos, "Player":name, "Club":club}
            
        except:
            pass
    # add the item to the data list using append() method
        data.append(item)
    # return the list of data
    return data